In [ ]:
# Question 5
import cv2
import numpy as np
import matplotlib.pyplot as plt

def foreground_histogram_equalization(image_path):
    # Read image and convert to HSV
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)
    
    # Create mask using saturation channel
    S_blur = cv2.GaussianBlur(S, (5, 5), 0)
    _, mask0 = cv2.threshold(S_blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask0, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    # Extract foreground and compute histogram
    fg_V = cv2.bitwise_and(V, V, mask=mask)
    hist = cv2.calcHist([fg_V], [0], mask, [256], [0, 256]).ravel()
    cdf = np.cumsum(hist)
    
    # Equalize foreground
    cdf_min = cdf[hist > 0][0] if np.any(hist > 0) else 0
    N = cdf[-1] if cdf[-1] > 0 else 1
    eq_lut = np.clip(np.round((cdf - cdf_min) / (N - cdf_min) * 255), 0, 255).astype(np.uint8)
    
    V_eq = V.copy()
    V_eq[mask > 0] = eq_lut[V[mask > 0]]
    
    # Combine with background
    result_hsv = cv2.merge([H, S, V_eq])
    result_rgb = cv2.cvtColor(result_hsv, cv2.COLOR_HSV2RGB)
    
    return img_rgb, result_rgb, H, S, V, mask, fg_V, V_eq, hist, cdf

# Main execution
image_path = 'E:/UoM MSc in AI/Semester 3/IT5437 - Computer Vision/Assignment/a1images/jeniffer.jpg'

try:
    original, result, H, S, V, mask, fg_V, V_eq, hist, cdf = foreground_histogram_equalization(image_path)
    
    # COMPARISON ANALYSIS
    foreground_pixels = V[mask > 0]
    foreground_eq_pixels = V_eq[mask > 0]
    background_pixels = V[mask == 0]
    
    # Display results with comparison
    plt.figure(figsize=(20, 12))
    
    # Row 1: Images
    plt.subplot(3, 4, 1)
    plt.imshow(original)
    plt.title('Original Image', fontweight='bold')
    plt.axis('off')
    
    plt.subplot(3, 4, 2)
    plt.imshow(mask, cmap='gray')
    plt.title('Foreground Mask', fontweight='bold')
    plt.axis('off')
    
    plt.subplot(3, 4, 3)
    plt.imshow(result)
    plt.title('Enhanced Result', fontweight='bold')
    plt.axis('off')
    
    plt.subplot(3, 4, 4)
    plt.imshow(S, cmap='gray')
    plt.title('Saturation Channel', fontweight='bold')
    plt.axis('off')
    
    # Row 2: Histograms
    plt.subplot(3, 4, 5)
    plt.hist(V.ravel(), bins=50, alpha=0.7, color='blue', label='Original')
    plt.hist(V_eq.ravel(), bins=50, alpha=0.7, color='red', label='Enhanced')
    plt.title('Full Image Histogram Comparison', fontweight='bold')
    plt.xlabel('Intensity')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(3, 4, 6)
    plt.hist(foreground_pixels, bins=50, alpha=0.7, color='green', label='Original FG')
    plt.hist(foreground_eq_pixels, bins=50, alpha=0.7, color='orange', label='Enhanced FG')
    plt.title('Foreground Histogram Comparison', fontweight='bold')
    plt.xlabel('Intensity')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(3, 4, 7)
    plt.hist(background_pixels, bins=50, alpha=0.7, color='purple', label='Background')
    plt.title('Background Histogram (Unchanged)', fontweight='bold')
    plt.xlabel('Intensity')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(3, 4, 8)
    plt.plot(cdf, 'r-', linewidth=2)
    plt.title('Cumulative Distribution Function', fontweight='bold')
    plt.xlabel('Intensity')
    plt.ylabel('Cumulative Frequency')
    plt.grid(True, alpha=0.3)
    
    # Row 3: Statistical comparison
    plt.subplot(3, 4, 9)
    metrics = ['Mean', 'Std Dev', 'Dynamic Range']
    original_stats = [np.mean(foreground_pixels), np.std(foreground_pixels), np.ptp(foreground_pixels)]
    enhanced_stats = [np.mean(foreground_eq_pixels), np.std(foreground_eq_pixels), np.ptp(foreground_eq_pixels)]
    
    x = np.arange(len(metrics))
    width = 0.35
    plt.bar(x - width/2, original_stats, width, label='Original FG', alpha=0.8)
    plt.bar(x + width/2, enhanced_stats, width, label='Enhanced FG', alpha=0.8)
    plt.title('Foreground Statistics Comparison', fontweight='bold')
    plt.xticks(x, metrics)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Intensity transformation curve
    plt.subplot(3, 4, 10)
    x_vals = np.arange(256)
    plt.plot(x_vals, x_vals, 'k--', alpha=0.5, label='Identity')
    plt.plot(x_vals, hist, 'b-', alpha=0.7, label='Original Histogram')
    plt.title('Intensity Distribution', fontweight='bold')
    plt.xlabel('Intensity')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Before-After comparison
    plt.subplot(3, 4, 11)
    plt.imshow(fg_V, cmap='gray')
    plt.title('Original Foreground', fontweight='bold')
    plt.axis('off')
    
    plt.subplot(3, 4, 12)
    plt.imshow(V_eq, cmap='gray')
    plt.title('Enhanced Foreground', fontweight='bold')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Comparison results
    print("\n" + "=" * 60)
    print("COMPARISON RESULTS")
    print("=" * 60)
    
    print("\n(a) ✓ Image split into HSV planes")
    print("(b) ✓ Foreground mask created using saturation thresholding")
    print("(c) ✓ Foreground extracted and histogram computed") 
    print("(d) ✓ Cumulative sum obtained using np.cumsum")
    print("(e) ✓ Foreground equalized using histogram equalization")
    print("(f) ✓ Background preserved and combined with enhanced foreground")
    
    print(f"\n{'Metric':<15} {'Original FG':<12} {'Enhanced FG':<12} {'Change':<10} {'Improvement':<12}")
    print("-" * 65)
    
    stats = [
        ('Mean', np.mean(foreground_pixels), np.mean(foreground_eq_pixels)),
        ('Std Dev', np.std(foreground_pixels), np.std(foreground_eq_pixels)),
        ('Dynamic Range', np.ptp(foreground_pixels), np.ptp(foreground_eq_pixels)),
        ('Contrast', np.std(foreground_pixels)/np.mean(foreground_pixels), 
         np.std(foreground_eq_pixels)/np.mean(foreground_eq_pixels))
    ]
    
    for metric, orig, enh in stats:
        change = enh - orig
        improvement = f"{(enh/orig-1)*100:+.1f}%" if orig != 0 else "N/A"
        print(f"{metric:<15} {orig:<12.2f} {enh:<12.2f} {change:>+8.2f} {improvement:>12}")
    
    print(f"\nKey Improvements:")
    print(f"- Contrast increased by {(stats[3][2]/stats[3][1]-1)*100:+.1f}%")
    print(f"- Foreground details enhanced while background remains natural")
    print(f"- Better intensity distribution in foreground regions")
    print(f"- Professional studio-quality enhancement")
    
except Exception as e:
    print(f"Error: {e}")